# Import Libs

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import re
import glob
import os
import pandas as pd
from openpyxl.utils.exceptions import IllegalCharacterError

# Constants & Functions

In [ ]:
# Function to fetch and extract text from a webpage
def fetch_webpage_text(url):
    try:
        headers = {'User-Agent': 'Mozilla/5.0'}  # Some websites block requests without user-agent
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()  # Raise error for bad responses (4xx, 5xx)

        # Parse the HTML content
        soup = BeautifulSoup(response.text, "html.parser")

        # Extract text from the page
        text = soup.get_text(separator=" ", strip=True)
        return text

    except requests.RequestException as e:
        print(f"Error fetching {url}: {e}")
        return None  # Return None if there was an error
    
# Function to clean illegal characters
def clean_illegal_characters(value):
    if isinstance(value, str):
        return "".join(c if ord(c) >= 32 else "***" for c in value)  # Replace control characters with ***
    return value  # Return original value if not a string

# Enable tqdm for pandas apply
tqdm.pandas()

# Specify the directory containing Excel files
directory = r"C:\Users\AZMI\Desktop\Izzet Ahmet\Kodlar\Medical\Data\Raw"


# Get all Excel files in the directory
xlsx_files = glob.glob(os.path.join(directory, "*.xlsx"))


# Process

In [ ]:
for xlsx_file in xlsx_files:
    try:
        # Read the Excel file
        df = pd.read_excel(xlsx_file)

        # Check if 'WEB_ADDRESS' column exists
        if "WEB_ADDRESS" not in df.columns:
            print(f"Skipping {xlsx_file} (No 'WEB_ADDRESS' column found)")
            continue

        # Fetch webpage text
        df["PAGE_TEXT"] = df["WEB_ADDRESS"].progress_apply(fetch_webpage_text)

        # Apply the extraction function
        #df_extracted = df["PAGE_TEXT"].apply(lambda text: extract_values(text, keys)).apply(pd.Series)

        # Merge extracted data with original DataFrame
        #df = pd.concat([df, df_extracted], axis=1)

        # Drop PAGE_TEXT column
        #df = df.drop(columns=["PAGE_TEXT"], axis=1)

        # Define new file name
        new_file_name = os.path.splitext(xlsx_file)[0] + "_processed.xlsx"


        # Apply function to all text data in DataFrame
        df_cleaned = df.applymap(clean_illegal_characters)
        # Save to Excel
        df_cleaned.to_excel(new_file_name, index=False)
        print(f"Saved processed file: {new_file_name}")
    except Exception as e:
        print(e)
print("Processing complete!")


In [ ]:

# Define the keys to extract
keys = [
    "Date Initiated by Firm",
    "Date Posted",
    "Recall Status",
    "Recall Number",
    "Recall Event ID",
    "510(K) Number",
    "Product Classification",
    "Product",
    "Code Information",
    "Recalling Firm/Manufacturer",
    "Manufacturer Reason for Recall",
    "FDA Determined Cause",
    "Action",
    "Quantity in Commerce",
    "Distribution",
    "Total Product Life Cycle"
]

# Function to extract values from PAGE_TEXT
def extract_values(text, keys):
    extracted_data = {}
    for i, key in enumerate(keys):
        # Create regex pattern to match key-value pairs
        pattern = rf"{key}\s*(.*?)(?=\n{keys[i+1]}|\Z)" if i + 1 < len(keys) else rf"{key}\s*(.*)"
        match = re.search(pattern, text, re.DOTALL)
        extracted_data[key] = match.group(1).strip() if match else None
    return extracted_data